<a href="https://colab.research.google.com/github/k-washi/python-cookbook/blob/master/ml/1/preprocessingML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from io import StringIO

csv_data = '''A, B, C, D
              1., 2., 3., 4.
              5., 6., , 8.
              10., 11., 12., '''

df = pd.read_csv(StringIO(csv_data))

In [4]:
print(df)

      A     B     C    D
0   1.0   2.0    3.   4.
1   5.0   6.0         8.
2  10.0  11.0   12.     


In [5]:
print(df.isnull().sum())

A     0
 B    0
 C    0
 D    0
dtype: int64
